In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.feature_selection import chi2, f_regression, f_classif, mutual_info_classif, mutual_info_regression 
from sklearn.feature_selection import SelectKBest, SelectFromModel, VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn import model_selection as ms
from sklearn.model_selection import cross_val_score, cross_val_predict

sklearn.set_config(print_changed_only=False)

import xgboost as xgb

from matplotlib import pyplot as plt

import import_ipynb
from Self_Written_Functions_Sheet import col_obj_list, nan_col_list, cols_nan_unique, nan_col_obj_list_fillna, fill_masvnrtype
from Self_Written_Functions_Sheet import rmse, conversion

%store -r

importing Jupyter notebook from Self_Written_Functions_Sheet.ipynb


In [2]:
# import dataset

hp_validation = pd.read_csv('../test.csv')
hp_validation

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,Comp

# Basic EDA and Imputations

In [3]:
# adding the number of bathrooms
hp_validation['TotalBaths'] = hp_validation['FullBath'] + hp_validation['HalfBath'] * 0.5

# adding the number of bathrooms in the basement
hp_validation['BsmtBaths'] = hp_validation['BsmtFullBath'] + hp_validation['BsmtHalfBath'] * 0.5

# adding the total number of finished SF of the basement
hp_validation['BsmtFinTotalSF'] = hp_validation['BsmtFinSF1'] + hp_validation['BsmtFinSF2']

# determining if there is unfinished SF in the basement
hp_validation['BsmtUnfin'] = [1 if x > 0 else 0 for x in hp_validation['BsmtUnfSF']]

# determining if a house has a 2nd floor
hp_validation['2ndFlr'] = [1 if x > 0 else 0 for x in hp_validation['2ndFlrSF']]

# determining if a house has any low quality finishes
hp_validation['LowQualFin'] = [1 if x > 0 else 0 for x in hp_validation['LowQualFinSF']]


# https://www.fixr.com/comparisons/porch-vs-deck#cQ
    # decks are diff than porches, but it seems that porches can be grouped
# adding the total SF of porches
hp_validation['TotalPorchSF'] = hp_validation['OpenPorchSF'] + hp_validation['EnclosedPorch'] + hp_validation['3SsnPorch'] + hp_validation['ScreenPorch']

# determining if there are any porches at all
hp_validation['Porches'] = [1 if x > 0 else 0 for x in hp_validation['TotalPorchSF']]

# determining if there are any decks at all
hp_validation['Decks'] = [1 if x > 0 else 0 for x in hp_validation['WoodDeckSF']]

# calculating the time between remodeling and selling the house
hp_validation['YrsRemodToSold'] = hp_validation['YrSold'] - hp_validation['YearRemodAdd']

# dropping some columns to save space
hp_val_myversion = hp_validation.drop(['BsmtFinSF1',
                                       'BsmtFinSF2',
                                       'OpenPorchSF',
                                       'EnclosedPorch',
                                       '3SsnPorch',
                                       'LowQualFinSF',
                                       'WoodDeckSF'], axis=1)

hp_validation

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,1.0,0.0,612.0,1,0,0,120,1,1,49
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,1.5,0.0,923.0,1,0,0,36,1,1,52
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,2.5,0.0,791.0,1,1,0,34,1,1,12
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,2.5,0.0,602.0,1,1,0,36,1,1,12
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,2.0,0.0,263.0,1,0,0,226,1,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,1.5,0.0,0.0,1,1,0,0,0,0,36
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,1.5,0.0,252.0,1,1,0,24,1,0,36
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960

In [4]:
# columns that have dtype=='object' vs not
cols_obj = []
cols_num = []
# Creates list of column names with dtype=='object' and one where dtype!='object'
col_obj_list(hp_validation, hp_validation.columns, cols_obj, cols_num)


# columns with object dtype that have null values
    # to use with list_nan_col
cols_w_nan_obj = []
# Outputs a list of the columns that have null values
nan_col_list(hp_validation, cols_obj, cols_w_nan_obj)


# columns with non-object dtype that have null values
cols_w_nan_num = []
# Outputs a list of the columns that have null values
nan_col_list(hp_validation, cols_num, cols_w_nan_num)

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'BsmtBaths',
 'BsmtFinTotalSF']

In [5]:
cols_w_nan_obj

['MSZoning',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [6]:
# Creating dataframes that display the values in our dataset
data_dict_obj = {}
data_dict_null_obj = {}
cols_nan_unique(hp_validation, cols_w_nan_obj, data_dict_obj, data_dict_null_obj)

# to visualize via dataframes the unique values that exist in the dataset
# to visualize the null count next to those values

data_dict_obj_df = pd.DataFrame.from_dict(data_dict_obj, orient='index').fillna('')
data_dict_null_obj_df = pd.DataFrame(data_dict_null_obj.items())
data_dict_null_obj_df.columns = ['Col_Name', 'Null_Count']
data_dict_null_obj_df = data_dict_null_obj_df.set_index('Col_Name')
pd.concat([data_dict_obj_df, data_dict_null_obj_df], axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Null_Count
MSZoning,RH,RL,RM,FV,C (all),,,,,,,,,,,4
Alley,Pave,Grvl,,,,,,,,,,,,,,1352
Utilities,AllPub,,,,,,,,,,,,,,,2
Exterior1st,VinylSd,Wd Sdng,HdBoard,Plywood,MetalSd,CemntBd,WdShing,BrkFace,AsbShng,BrkComm,Stucco,AsphShn,CBlock,,,1
Exterior2nd,VinylSd,Wd Sdng,HdBoard,Plywood,MetalSd,Brk Cmn,CmentBd,ImStucc,Wd Shng,AsbShng,Stucco,CBlock,BrkFace,AsphShn,Stone,1
MasVnrType,None,BrkFace,Stone,BrkCmn,,,,,,,,,,,,16
BsmtQual,TA,Gd,Ex,Fa,,,,,,,,,,,,44
BsmtCond,TA,Po,Fa,Gd,,,,,,,,,,,,45
BsmtExposure,No,Gd,Mn,Av,,,,,,,,,,,,44
BsmtFinType1,Rec,ALQ,GLQ,Unf,BLQ,LwQ,,,,,,,,,,42


In [7]:
data_dict_null_obj_df

,Null_Count
Col_Name,
MSZoning,4
Alley,1352
Utilities,2
Exterior1st,1
Exterior2nd,1
MasVnrType,16
BsmtQual,44
BsmtCond,45
BsmtExposure,44


In [8]:
hp_validation[hp_validation['Alley'].isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,1.0,0.0,612.0,1,0,0,120,1,1,49
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,1.5,0.0,923.0,1,0,0,36,1,1,52
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,2.5,0.0,791.0,1,1,0,34,1,1,12
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,2.5,0.0,602.0,1,1,0,36,1,1,12
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,2.0,0.0,263.0,1,0,0,226,1,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,1.5,0.0,0.0,1,1,0,0,0,0,36
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,1.5,0.0,252.0,1,1,0,24,1,0,36
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960

In [9]:
# dropping features with high NaN count from train set
hp_val_myversion = hp_val_myversion.drop(['Alley',
                                          'FireplaceQu',
                                          'PoolQC',
                                          'Fence',
                                          'MiscFeature'], axis=1)

In [10]:
# Columns with dtype Object that should NOT have NaNs:
should_not_have_nan_obj = ['MasVnrType', 'Electrical']

# Columns with dtype non-Object that should NOT have NaNs:
should_not_have_nan_num = ['LotFrontage', 'MasVnrArea']

# Replace Appropriate Columns' NaNs with 'DNE'
obj_DNE_list = [i for i in cols_w_nan_obj if i not in should_not_have_nan_obj]
num_DNE_list = [i for i in cols_w_nan_num if i not in should_not_have_nan_num]

# from Self_Written_Functions_Sheet_Recover
    # Inputs DNE or 0 for null values based on column's data type
nan_col_obj_list_fillna(hp_validation, obj_DNE_list)

# from Self_Written_Functions_Sheet_Recover
    # Inputs DNE or 0 for null values based on column's data type
nan_col_obj_list_fillna(hp_validation, num_DNE_list)

In [11]:
# outputs columns that contain NaNs
cols_w_nan_obj2 = []

nan_col_list(hp_validation, cols_w_nan_obj, cols_w_nan_obj2)

['MasVnrType']

In [12]:
# outputs columns that contain NaNs
cols_w_nan_num2 = []

nan_col_list(hp_validation, cols_w_nan_num, cols_w_nan_num2)

['LotFrontage', 'MasVnrArea']

In [13]:
# filling the missing value with the mode
hp_validation['Electrical'].fillna(hp_validation['Electrical'].mode()[0], inplace=True)
hp_validation[hp_validation['Electrical'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold


In [14]:
data_dict_num = {}
data_dict_null_num = {}

# From Self_Written_Functions_Sheet_Recover
# Creates 2 dictionaries:
#     1st: contains unique values of the column
#     2nd: contains the null count
cols_nan_unique(hp_validation, cols_w_nan_num, data_dict_num, data_dict_null_num)

# creating a dataframe from the dictionaries
data_dict_null_num_df = pd.DataFrame(data_dict_null_num.items())
data_dict_null_num_df.columns = ['Col_Name', 'Null_Count']
data_dict_null_num_df = data_dict_null_num_df.set_index('Col_Name')
data_dict_null_num_df

,Null_Count
Col_Name,
LotFrontage,227
MasVnrArea,15
BsmtFinSF1,0
BsmtFinSF2,0
BsmtUnfSF,0
TotalBsmtSF,0
BsmtFullBath,0
BsmtHalfBath,0
GarageYrBlt,0


In [15]:
# doing the following just in case

# converting the masvnrtype to 0 if NaN
hp_validation.loc[hp_validation.MasVnrArea.isna(), 'MasVnrArea'] = 0
hp_validation[hp_validation['MasVnrArea'].isna()]

# converting the masvnrtype to 'None' if NaN
hp_validation.loc[(hp_validation.MasVnrType.isna()), 'MasVnrType']='None'

# Will use mode of MasVnrType if MasVnrArea is > 1
# Will assume MasVnrArea of 1 are actually 0 based on previous cell
hp_validation.loc[hp_validation.MasVnrArea==1, 'MasVnrArea'] = 0

In [16]:
# ['MasVnrType'] should = the mode of the ['MasVnrType'] that is NOT None
from collections import Counter
masvnrtype_mode = [x[0] for x in Counter(list(hp_validation['MasVnrType'])).most_common()]
    # outputs something that looks like: ['None', 'BrkFace', 'Stone', 'BrkCmn']
    
# if there is a 'None' value in the list created and MasVnrArea is greater than 1,
    # use the next value to skip 'None' and assign the mode
if masvnrtype_mode[0]=='None':
    hp_validation.loc[(hp_validation.MasVnrArea > 1) & (hp_validation.MasVnrType == 'None'),
                      'MasVnrType'] = masvnrtype_mode[1]
else:
    hp_validation.loc[(hp_validation.MasVnrArea > 1) & (hp_validation.MasVnrType == 'None'),
                      'MasVnrType'] = masvnrtype_mode[0]

In [17]:
# getting the list that contain the prerequisites
mvt_list = set(list(hp_validation[(hp_validation['MasVnrType']!='None') & (hp_validation['MasVnrArea']==0)]['MasVnrType']))

# from Self_Written_Functions_Sheet_Recover
    # The function replaces the 0 in the 'MasVnrArea' to the mode
    # of the 'MasVnrArea' based on the corresponding 'MasVnrType'
fill_masvnrtype(hp_validation, mvt_list)

# the 2nd object printed is the average mode

[0]
176.0


In [18]:
cols_w_nan_obj2 = []

nan_col_list(hp_validation, cols_w_nan_obj, cols_w_nan_obj2)

[]

In [19]:
cols_w_nan_num2 = []

nan_col_list(hp_validation, cols_w_nan_num, cols_w_nan_num2)

['LotFrontage']

## Lot Frontage Imputation using KNN Imputer

In [20]:
# dummify the variables
hp_val_dum = pd.get_dummies(hp_validation, drop_first=True)

# one hot encoding for mssubclass since it is numeric
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='first')
enc.fit(hp_val_dum[['MSSubClass']])
oh_labels = enc.transform(hp_val_dum[['MSSubClass']]).toarray()
mssubcl_val_dum = pd.DataFrame(oh_labels)
mssubcl_val_dum.columns = enc.get_feature_names(['MSSubClass'])

# Concatenate the dataframes and drop Id and original MSSubClass
hp_val_dum = pd.concat([hp_val_dum, mssubcl_val_dum], axis=1, sort=False)
hp_val_dum = hp_val_dum.drop(['Id', 'MSSubClass'], axis=1)

  
# Get list of columns that have dtype != object
cols_obj_val_dum = []
cols_num_val_dum = []

'''
the function below creates a list of column names with dtype=='object'
and a list of column names with dtype!='object'
'''
col_obj_list(hp, hp.columns, cols_obj_val_dum, cols_num_val_dum)


'''
we are removing Id and MSSubclass
    - Id because it has no bearing on the result
    - MSSubClass because it is categorical
'''
to_remove = ['Id', 'MSSubClass', 'LotFrontage', 'Decks',
             'Porches', 'LowQualFin', '2ndFlr', 'BsmtUnfin']
cols_num_val_dum = [x for x in cols_num_val_dum if x not in to_remove]
cols_num_val_dum

# Create DF of dtype != Object for standardization
hp_num_val_dum = hp[cols_num_val_dum]

### Standardization Process

In [21]:
from sklearn.preprocessing import StandardScaler

# Standardize numerical features
strdscaler = StandardScaler()
standardized = strdscaler.fit_transform(hp_num_val_dum)
hp_num_val = pd.DataFrame(standardized, columns = hp_num_val_dum.columns)

# drop columns in hp_val_dum to prevent duplicate features during concatenation
hp_val_dum = hp_val_dum.drop(cols_num_val_dum, axis=1)

# Concatenate the dummified df and the standardized df
hp_val_strd_nolf = pd.concat([hp_num_val, hp_val_dum], axis=1, sort=False)

### Running KNN Imputer then Standardize 'LotFrontage'

In [23]:
# creating my version for KNN Imputer
hp_val_myver_nolf = hp_val_strd_nolf.copy()

# run knn imputer
root_n = int(np.sqrt(len(hp_val_strd_nolf.columns)))
knn = KNNImputer(n_neighbors=root_n)

# we can run KNN Imputer on the whole dataset bc there are no null values in the other columns
knn_lf = knn.fit_transform(hp_val_strd_nolf)

hp_val_strd = pd.DataFrame(knn_lf, columns=hp_val_strd_nolf.columns)

hp_val_strd.isna().values.any()

# standardize lot frontage
strdscaler = StandardScaler()
standardized = strdscaler.fit_transform(hp_val_strd[['LotFrontage']])

hp_val_strd['LotFrontage'] = pd.DataFrame(standardized, columns=['LotFrontage'])

### Running KNN Imputer for MyVersion of the dataset

In [24]:
# removing columns for myversion

# with inspiration from:
# https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe

cols = ['YearBuilt', 'HouseStyle', 'OverallQual', 'ExterQual',
        'BsmtQual', 'GarageArea', 'GarageQual', 'BsmtFinSF1',
        'BsmtFinSF2', 'OpenPorchSF', 'EnclosedPorch',
        '3SsnPorch', 'LowQualFinSF', 'WoodDeckSF', 'Alley',
        'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

for col in cols:
    hp_val_myver_nolf = hp_val_myver_nolf.loc[:, ~hp_val_myver_nolf.columns.str.contains(col)]

# run knn imputer
root_n = int(np.sqrt(len(hp_val_strd_nolf.columns)))
knn = KNNImputer(n_neighbors=root_n)
knn_lf = knn.fit_transform(hp_val_myver_nolf)

hp_val_myver = pd.DataFrame(knn_lf, columns=hp_val_myver_nolf.columns)

hp_val_myver.isna().values.any()

# Standardize 'LotFrontage'
strdscaler = StandardScaler()
standardized = strdscaler.fit_transform(hp_val_myver[['LotFrontage']])

hp_val_myver['LotFrontage'] = pd.DataFrame(standardized, columns=['LotFrontage'])
hp_val_myversion = hp_val_myver.copy()
hp_val_myversion

,LotArea,OverallCond,YearRemodAdd,MasVnrArea,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalBaths,BsmtBaths,BsmtFinTotalSF,TotalPorchSF,YrsRemodToSold,LotFrontage,BsmtUnfin,2ndFlr,LowQualFin,Porches,Decks,MSZoning_DNE,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_DNE,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_DNE,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_DNE,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_DNE,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_DNE,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_DNE,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_DNE,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_DNE,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190
0,-0.203934,-0.517649,0.880362,0.523014,-0.943795,-0.473766,-0.814309,1.167626,0.393013,1.113886,-0.241236,0.793546,1.229699,0.163894,-0.211605,0.921812,-0.952231,0.296262,0.313159,-0.270407,-0.063709,-0.087748,-1.601578,0.138375,1.174852,1.066169,0.485432,-0.245576,-0.871676,0.528811,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [25]:
hp_val_strd

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalBaths,BsmtBaths,BsmtFinTotalSF,TotalPorchSF,YrsRemodToSold,LotFrontage,BsmtUnfin,2ndFlr,LowQualFin,Porches,Decks,MSZoning_DNE,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_DNE,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_DNE,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_DNE,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_DNE,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_DNE,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_DNE,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_DNE,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Gd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_DNE,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_30,MSSubClass_40,MSSubClass_45

In [32]:
# getting lasso reduced features from ML_HousingPrice_ModelTraining_LinearRegressions

cols_strd = hp_val_strd.columns
cols_to_use = hp_lasso_strd_reduced.columns

'''
looking for discrepancies in the new list compared to the list of feature names obtained
from running lasso for training set
'''
missing_cols = [x for x in cols_to_use if x not in cols_strd]
missing_cols

['GarageQual_Ex']

In [35]:
# since the column is missing from the new list, we can assume that the values would be 0
for col in missing_cols:
    hp_val_strd[col] = 0.0

In [36]:
hp_val_lasso = hp_val_strd[cols_to_use]
hp_val_lasso

,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,GrLivArea,BsmtFullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,ScreenPorch,TotalBaths,BsmtFinTotalSF,TotalPorchSF,YrsRemodToSold,LotFrontage,LowQualFin,Decks,MSZoning_FV,MSZoning_RL,Street_Pave,LotShape_IR2,LotShape_Reg,LotConfig_CulDSac,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Somerst,Neighborhood_StoneBr,Condition1_Norm,Condition1_RRAe,BldgType_Twnhs,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_Wd Sdng,Exterior2nd_Stucco,MasVnrType_Stone,ExterQual_Fa,ExterQual_TA,ExterCond_Fa,ExterCond_TA,Foundation_PConc,BsmtQual_Ex,BsmtQual_TA,BsmtCond_Fa,BsmtExposure_Gd,BsmtExposure_No,BsmtFinType1_GLQ,BsmtFinType2_BLQ,Heating_Grav,HeatingQC_TA,CentralAir_Y,KitchenQual_TA,Functional_Maj2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Gd,GarageType_Basment,GarageType_CarPort,GarageQual_Ex,GarageQual_Gd,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,PoolQC_Gd,Fence_GdWo,SaleType_ConLI,SaleType_New,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_30,MSSubClass_160
0,-0.203934,0.658506,-0.517649,1.052959,0.523014,0.617283,-0.473766,-0.814309,0.393013,1.113886,1.229699,-0.211605,0.921812,-0.952231,0.296262,0.313159,0.357973,-0.750831,-0.270407,1.174852,0.485432,-0.245576,-0.871676,0.538349,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.087252,-0.068293,2.177825,0.158428,-0.571872,1.245719,0.504925,0.277348,-0.489391,-0.819502,-0.760202,-0.211605,-0.316329,0.605965,0.236767,0.313159,-0.056795,1.627328,-0.270407,0.386571,1.084079,-0.827879,0.388660,0.586736,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.080162,0.658506,-0.517649,0.986698,0.333085,0.108989,-0.319490,-0.642225,0.542706,1.113886,1.229699,-0.211605,-0.316329,0.605965,0.291855,0.313159,0.640770,-0.750831,-0.270407,1.174852,0.001232,-0.426949,-0.823201,0.248027,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.092325,0.658506,-0.517649,-1.862551,-0.571872,-0.514826,-0.714823,-0.531984,0.406800,1.113886,-0.760202,-0.211605,0.302742,0.605965,0.285244,1.652119,0.801022,-0.750831,-0.270407,-1.189990,-0.593014,2.102728,0.631032,0.441575,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.385566,1.385305,-0.517649,0.953567,1.383282,0.499451,0.222888,-0.037243,1.354202,1.113886,1.229699,-0.211605,1.540882,0.605965,0.289651,1.652119,1.715398,0.781406,-0.270407,1.174852,0.373185,-0.026019,-0.726252,-1.251971,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Running Predictions